In [1]:
import scipy.signal
import numpy as np
import dask
import dask.array as da
from dask.distributed import Client

In [4]:
from dask.distributed import Client
client = Client(n_workers=4, threads_per_worker=4, processes=True, memory_limit='2.5GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 9.31 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52908,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 9.31 GiB
Comm: tcp://127.0.0.1:52929,Total threads: 4
Dashboard: http://127.0.0.1:52935/status,Memory: 2.33 GiB
Nanny: tcp://127.0.0.1:52911,


In [5]:
class GameOfLife:
    def __init__(self, shape=(3, 3)):
        # Initialize the GameofLife instance
        self.shape = shape
        self.kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]], dtype=int)
        self.state = np.zeros(self.shape, dtype=bool)

    def load_grid_from_file(self, input_file):
        # Load the initial state from a text file
        with open(input_file, "r") as f:
            width, height = map(int, f.readline().split())
            self.shape = (height, width)
            self.state = np.zeros(self.shape, dtype=bool)
            for line in f:
                value = line.split()
                if len(value) >= 2:
                    y, x = map(int, value[:2])
                    if 0 <= y < height and 0 <= x < width:
                        self.state[y, x] = 1
        return self.state

    def update_state(self, state):
        # Update the state for the next generation using Conway's Game of Life rules
        conv_state = scipy.signal.convolve2d(state, self.kernel, mode="same")
        temp_state = np.zeros_like(state)
        temp_state[(conv_state < 2) | (conv_state > 3)] = 0
        temp_state[((conv_state == 2) | (conv_state == 3)) & (state == 1)] = 1
        temp_state[conv_state == 3] = 1
        return temp_state

    def save_state_to_file(self, output_file, state):
        # Save the current state to a text file
        with open(output_file, "w") as f:
            f.write(f"{state.shape[1]} {state.shape[0]}\n")
            for y in range(state.shape[0]):
                for x in range(state.shape[1]):
                    if state[y, x] == 1:
                        f.write(f"{y} {x}\n")
        return True  # Indicate success


def run_game(input_name, output_name, generations, chunksize):
    # Load the grid from the file
    game = GameOfLife()
    game.load_grid_from_file(input_name)
    grid_dask = da.from_array(game.state, chunks=(chunksize[0], chunksize[1]))
    for _ in range(generations):
        grid_dask = grid_dask.map_overlap(game.update_state, depth=1, boundary="none")
    grid = grid_dask.compute()

    game.save_state_to_file(output_name, grid)
    return grid

In [6]:
dask.config.set(scheduler='processes')
if __name__ == "__main__":
    input_name = "./Data/10000x10000_0.1.txt"
    output_name = "New_Dask_output2.txt"
    generations = 1
    chunksize = (1000, 1000)
    grid = run_game(input_name, output_name, generations, chunksize)
client.shutdown()
print(grid)

C:\Users\User\anaconda3\Lib\site-packages\dask\base.py:1366: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
2023-10-21 22:10:55,065 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x00000238426E1810>>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\tornado\ioloop.py", line 919, in _run
    val = self.callback()
          ^^^^^^^^^^^^^^^
  File "C:\Users\User\anaconda3\Lib\site-packages\bokeh\server\tornado.py", line 779, in _keep_alive
    c.send_ping()
  File "C:\Users\User\anaconda3\Lib\site-packages\bokeh\server\connection.py", line 91, in send_ping
    self._socket.ping(str(self._ping_count).encode("utf-8"))
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\tornado\websocket.py", line 439, in ping
    raise WebSocketClosedError

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
